In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; 
 
# Importing sklearn libraries
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
 

 
from keras.utils import to_categorical #to create dummy variable
from keras.utils import np_utils
from keras.models import Sequential
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D


import import_ipynb
import utils



from sklearn.svm import SVC

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


importing Jupyter notebook from utils.ipynb


In [2]:
# SOME IMPORTANT GLOBAL VARIABLE
labels = ["Bedroom","Coast","Forest","HighWay","Industrial","InsideCity","Kitchen","LivingRoom","Mountain","Office","OpenCountry","Store","Street","Suburb","TallBuilding"]
train_dir = '../images/train/'
test_dir = '../images/test/'


# CREATION OF DATA--> test and train 


In [3]:
#LIST OF ISTRING WHICH REPRESENT ALL THE IMAGES!
list_of_images = utils.list_of_path(labels,train_dir)
train_data,train_labels = utils.read_and_process_images(list_of_images,dimension=224)
train_labels_dummy  = to_categorical(train_labels,15)

In [4]:
list_of_images_test = utils.list_of_path(labels,test_dir)
test_data,test_labels = utils.read_and_process_images(list_of_images_test,dimension=224)
test_labels_dummy  = to_categorical(test_labels,15)

#  model choice (VGG16) and feature extraction 

In [5]:
model = VGG16(weights='imagenet', include_top=False)


In [ ]:
model.summary()

In [9]:
# define a function which extract feature for all my train images

def feature_extraction(dataset,pre_model):
    res = []
    cont = 0
    #viaggio nel dataset
    for path in dataset:
        print(cont)
        cont = cont+1
        im = load_img(path, target_size=(224, 224))
        im = img_to_array(im)
        im = np.expand_dims(im, axis=0)
        im = imagenet_utils.preprocess_input(im)
        res.append(im)
    images = np.vstack(res)
    features = pre_model.predict(images, batch_size=64)
    features_flatten = features.reshape((features.shape[0], 7 * 7 * 512))
    return images, features, features_flatten     

In [ ]:
train_images,feature_matrix,feature_array= feature_extraction(list_of_images,model)

In [ ]:
test_images,feature_matrix_test,feature_array_test = feature_extraction(list_of_images_test,model)

In [13]:
def predict_with_linear_SVM(features_array,feature_lab,feature_array_test,test_labels,num_classes = 15):
    classif = [SVC(kernel="linear") for _ in range(num_classes)] # array of classifier to be trained
    # one-vs-all approach with  
    curr_label = 0
    for clf in classif:
        print(curr_label)
        v = np.array([1 if label==curr_label else 0 for label in feature_lab])
        clf = clf.fit(features_array, v)
        curr_label = curr_label + 1
    # now we want to test 
    prediction = []
    for image in feature_array_test:
        pred = np.array([np.dot(clf.coef_,image) + clf.intercept_ for clf in classif])
        prediction.append(np.argmax(pred))
    prediction = np.asarray(prediction)
    #calculate accuracy
    cont=0
    for i in range(len(prediction)):
        if prediction[i]==test_labels[i]:
            cont = cont +1
        else:
            continue
        
    acc = cont/len(prediction)
    
    return classif,prediction,acc
                    
    
    

In [14]:
classifiers,prediction,accuracy = predict_with_linear_SVM(feature_array,train_labels,feature_array_test,test_labels)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [15]:
accuracy

0.8991624790619765

# UTILIZZO DAG PER IL MULTICLASS SVM 


In [245]:
prediction,accuracy = predict_with_SVM_with_Dag(feature_array,train_labels,feature_array_test,test_labels)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [246]:
accuracy

0.07738693467336684

In [241]:
len(prediction)

2500